# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [116]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')


from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [117]:
import os
print(os.getcwd())

/home/workspace


In [183]:
# load data from database
engine = create_engine('sqlite:////home/workspace/matheus_project.db')
df = pd.read_sql("SELECT * FROM ETL_part", engine)
X = df["message"]
Y = df.drop(columns=["id","original","genre","message"])

Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 1.1 Stop Words

In [119]:
# Normalize text

text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
print(text)

a comitee in delmas 19  rue   street   janvier  impasse charite  2  we have about 500 people in a temporary shelter and we are in dire need of water  food  medications  tents and clothes  please stop by and see us 


### 1.2 Test part of tokenize

In [120]:
# import statements
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize


token = word_tokenize(text)
print(token)

# tag each word with part of speech
pos_tag(token)

['a', 'comitee', 'in', 'delmas', '19', 'rue', 'street', 'janvier', 'impasse', 'charite', '2', 'we', 'have', 'about', '500', 'people', 'in', 'a', 'temporary', 'shelter', 'and', 'we', 'are', 'in', 'dire', 'need', 'of', 'water', 'food', 'medications', 'tents', 'and', 'clothes', 'please', 'stop', 'by', 'and', 'see', 'us']


[('a', 'DT'),
 ('comitee', 'NN'),
 ('in', 'IN'),
 ('delmas', 'NN'),
 ('19', 'CD'),
 ('rue', 'JJ'),
 ('street', 'NN'),
 ('janvier', 'NN'),
 ('impasse', 'VBZ'),
 ('charite', 'JJ'),
 ('2', 'CD'),
 ('we', 'PRP'),
 ('have', 'VBP'),
 ('about', 'IN'),
 ('500', 'CD'),
 ('people', 'NNS'),
 ('in', 'IN'),
 ('a', 'DT'),
 ('temporary', 'JJ'),
 ('shelter', 'NN'),
 ('and', 'CC'),
 ('we', 'PRP'),
 ('are', 'VBP'),
 ('in', 'IN'),
 ('dire', 'JJ'),
 ('need', 'NN'),
 ('of', 'IN'),
 ('water', 'NN'),
 ('food', 'NN'),
 ('medications', 'NNS'),
 ('tents', 'NNS'),
 ('and', 'CC'),
 ('clothes', 'NNS'),
 ('please', 'VBP'),
 ('stop', 'VB'),
 ('by', 'IN'),
 ('and', 'CC'),
 ('see', 'VB'),
 ('us', 'PRP')]

### 1.3 Lemmatizer

In [121]:
lemma = nltk.WordNetLemmatizer()
for i in token:
    print(lemma.lemmatize(i))

a
comitee
in
delmas
19
rue
street
janvier
impasse
charite
2
we
have
about
500
people
in
a
temporary
shelter
and
we
are
in
dire
need
of
water
food
medication
tent
and
clothes
please
stop
by
and
see
u


### 2. Write a tokenization function to process your text data

In [155]:
def tokenize(text):
    text =  re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    token = word_tokenize(text)
    lemma = nltk.WordNetLemmatizer()
    return [lemma.lemmatize(i).strip() for i in token]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [206]:
pipeline = Pipeline([('cvt', CountVectorizer(tokenizer=tokenize)),
                     ('tf', TfidfTransformer()),
                     ('model', MultiOutputClassifier(RandomForestClassifier())),])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [207]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state = 42)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('cvt', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [208]:
prediction = pipeline.predict(X_test)
prediction[:1]

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [210]:
enumareted = enumerate(y_test)
for i, j in enumareted:
    print('''Column: {} 
    {}'''.format(j,classification_report(y_test[j], prediction[:, i])))


Column: related 
                 precision    recall  f1-score   support

          0       0.62      0.33      0.44      2054
          1       0.81      0.94      0.87      6534
          2       0.58      0.30      0.39        64

avg / total       0.77      0.79      0.76      8652

Column: request 
                 precision    recall  f1-score   support

          0       0.89      0.98      0.93      7180
          1       0.83      0.38      0.52      1472

avg / total       0.88      0.88      0.86      8652

Column: offer 
                 precision    recall  f1-score   support

          0       1.00      1.00      1.00      8614
          1       0.00      0.00      0.00        38

avg / total       0.99      1.00      0.99      8652

Column: aid_related 
                 precision    recall  f1-score   support

          0       0.73      0.87      0.79      5107
          1       0.74      0.53      0.62      3545

avg / total       0.73      0.73      0.72      8652

C

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [223]:
parameters = {'cvt__min_df': [1, 3],
              'tf__use_idf':[True, False],
              'model__estimator__n_estimators':[15, 25], 
              'model__estimator__min_samples_split':[2, 5, 10]}

cv = GridSearchCV(pipeline, param_grid = parameters, verbose = 10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [224]:
tuned_model = cv.fit(X_train, y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] cvt__min_df=1, model__estimator__min_samples_split=2, model__estimator__n_estimators=15, tf__use_idf=True 
[CV]  cvt__min_df=1, model__estimator__min_samples_split=2, model__estimator__n_estimators=15, tf__use_idf=True, score=0.2201537147736977, total=  48.3s
[CV] cvt__min_df=1, model__estimator__min_samples_split=2, model__estimator__n_estimators=15, tf__use_idf=True 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   56.8s remaining:    0.0s


[CV]  cvt__min_df=1, model__estimator__min_samples_split=2, model__estimator__n_estimators=15, tf__use_idf=True, score=0.22117847993168233, total=  48.7s
[CV] cvt__min_df=1, model__estimator__min_samples_split=2, model__estimator__n_estimators=15, tf__use_idf=True 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.9min remaining:    0.0s


[CV]  cvt__min_df=1, model__estimator__min_samples_split=2, model__estimator__n_estimators=15, tf__use_idf=True, score=0.223436966176973, total=  49.2s
[CV] cvt__min_df=1, model__estimator__min_samples_split=2, model__estimator__n_estimators=15, tf__use_idf=False 


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  8.6min remaining:    0.0s


[CV]  cvt__min_df=1, model__estimator__min_samples_split=2, model__estimator__n_estimators=25, tf__use_idf=True, score=0.23009907755380937, total= 1.3min
[CV] cvt__min_df=1, model__estimator__min_samples_split=2, model__estimator__n_estimators=25, tf__use_idf=False 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 10.1min remaining:    0.0s


[CV]  cvt__min_df=1, model__estimator__min_samples_split=2, model__estimator__n_estimators=25, tf__use_idf=False, score=0.21861656703672075, total= 1.3min
[CV] cvt__min_df=1, model__estimator__min_samples_split=2, model__estimator__n_estimators=25, tf__use_idf=False 
[CV]  cvt__min_df=1, model__estimator__min_samples_split=2, model__estimator__n_estimators=25, tf__use_idf=False, score=0.22801024765157984, total= 1.3min
[CV] cvt__min_df=1, model__estimator__min_samples_split=2, model__estimator__n_estimators=25, tf__use_idf=False 
[CV]  cvt__min_df=1, model__estimator__min_samples_split=2, model__estimator__n_estimators=25, tf__use_idf=False, score=0.2350529552442774, total= 1.3min
[CV] cvt__min_df=1, model__estimator__min_samples_split=5, model__estimator__n_estimators=15, tf__use_idf=True 
[CV]  cvt__min_df=1, model__estimator__min_samples_split=5, model__estimator__n_estimators=15, tf__use_idf=True, score=0.21315115286080275, total=  41.7s
[CV] cvt__min_df=1, model__estimator__min_sa

[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed: 73.0min finished


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [225]:
prediction_improved = tuned_model.predict(X_test)
prediction_improved

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [226]:
enumareted = enumerate(y_test)
for i, j in enumareted:
    print('''Column: {} 
    {}'''.format(j,classification_report(y_test[j], prediction_improved[:, i])))

Column: related 
                 precision    recall  f1-score   support

          0       0.69      0.30      0.42      2054
          1       0.81      0.95      0.88      6534
          2       0.50      0.39      0.44        64

avg / total       0.78      0.80      0.77      8652

Column: request 
                 precision    recall  f1-score   support

          0       0.90      0.98      0.94      7180
          1       0.85      0.45      0.59      1472

avg / total       0.89      0.89      0.88      8652

Column: offer 
                 precision    recall  f1-score   support

          0       1.00      1.00      1.00      8614
          1       0.00      0.00      0.00        38

avg / total       0.99      1.00      0.99      8652

Column: aid_related 
                 precision    recall  f1-score   support

          0       0.77      0.85      0.81      5107
          1       0.75      0.63      0.68      3545

avg / total       0.76      0.76      0.76      8652

C

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [227]:
with open('model_disaster','wb') as f:
    pickle.dump(tuned_model,f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.